In [ ]:
import numpy as np
import pandas as pd
import requests
import re
import matplotlib.pyplot as plt


def pricing_derivatives_in_time(SOFR, URL, name, futures_contract_settle, futures_contract_expiration_date, 
                                spot_dividends=None):
    """
    futures_contract_settle = [41.902-.35, 41.902, 41.341, 41.6, 42.149, 42.83]
    futures_contract_expiration_date = pd.to_datetime("2025-12-29")
    """
    
    dates = SOFR.index

    SOFR_daily = (1 + SOFR/100)**(1/360) - 1 
    SOFR_daily

    spot = fetch_from_investing(URL)

    spot_price = spot.iloc[:,0]
    spot_price = spot_price[spot_price.index.isin(dates)]

    futures_contract_expiration_date = pd.to_datetime(futures_contract_expiration_date)
    futures_contract_days_to_exp = (futures_contract_expiration_date - dates[-1]).days
    futures_contract = pd.Series(futures_contract_settle, index=dates)


    futures_contract_days_to_exp = []
    for day in range(len(dates)):
        futures_contract_days_to_exp.append((futures_contract_expiration_date - dates[day]).days)
    futures_contract_days_to_exp = pd.Series(futures_contract_days_to_exp, index=dates)

    if spot_dividends is None:
        spot_dividends = pd.Series(np.zeros(len(dates)), index=dates)
    else :
        spot_dividends = spot_dividends[spot_dividends.index.isin(dates)]

    theoretical_price = spot_price * np.exp((SOFR_daily-spot_dividends) * futures_contract_days_to_exp)

    futures_contract_difference = futures_contract - theoretical_price
    futures_contract_risk_premium = futures_contract_settle / spot_price - 1

    spot = pd.DataFrame({
        "SOFR Rate": SOFR,
        "Horizon (days)": futures_contract_days_to_exp,
        f"{name} Price": spot_price,
        "Market Settlement": futures_contract,
        "Theoretical Price": theoretical_price,
        "Difference": futures_contract_difference,
        "Risk Premium": futures_contract_risk_premium
    })

    # plot the table for LaTeX
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')

    table_data = spot.reset_index() 
    table_data = table_data.rename(columns={"index": "Date"})
    table_data['Date'] = table_data['Date'].dt.date

    cell_text = table_data.round(4).values
    col_labels = table_data.columns

    the_table = ax.table(
        cellText=cell_text,
        colLabels=col_labels,
        loc='center',
        cellLoc='center'
    )
    the_table.auto_set_font_size(True)
    the_table.set_fontsize(10)
    the_table.scale(1.2, 1.8)

    # Apply styles to the table
    for key, cell in the_table.get_celld().items():
        cell.set_facecolor('#001a60')  # Set background color to blue
        cell.set_edgecolor('white')  # Set border color to white
        cell.set_text_props(color='white')  # Set font color to white

        # Make the Date and first column bold
        if key[1] == 0:  # Date column
            cell.set_text_props(weight='bold')
        # Make the first row (header) bold
        if key[0] == 0:  # Header row
            cell.set_text_props(weight='bold')

    # Adjust layout to reduce margins
    plt.tight_layout(pad=0.1)

    plt.savefig(f"latex/figures/{name}_derivatives.pdf", bbox_inches='tight', pad_inches=0.2)
    plt.show()

    # plot a bar chart of the difference between theoretical and market price
    plt.figure(figsize=(10, 5))
    plt.bar(spot.index.strftime('%m-%d'), spot['Difference'], color='#001a60')
    plt.xlabel('Date')
    plt.ylabel('Difference (Market - Theoretical)')
    plt.title('Difference between Market Settlement and Theoretical Price of futures_contract')
    plt.axhline(0, color='#880000', linestyle='-', linewidth=2)
    plt.grid(False)
    for i, v in enumerate(spot['Difference']):
        plt.text(i, v if v >= 0 else v - 0.006, f"{v:.2f}", color='black', ha='center', fontsize=8)
    plt.savefig(f"latex/figures/{name}_difference.pdf", bbox_inches='tight', pad_inches=0.2)
    plt.show()

    # plot the risk premium
    plt.figure(figsize=(10, 5))
    plt.bar(spot.index.strftime('%m-%d'), spot['Risk Premium']*100, color='#001a60')
    plt.xlabel('Date')
    plt.ylabel('Risk Premium (%)')
    plt.title('Risk Premium of futures_contract over Theoretical Price')
    plt.axhline(0, color='#880000', linestyle='-', linewidth=2)
    plt.grid(False)
    plt.savefig(f"latex/figures/{name}_risk_premium.pdf", bbox_inches='tight', pad_inches=0.2)
    plt.show()